## Neural Network on $4 \times 4$ downsampled MNIST Digits  
Sebastian Molina  
smolinad@unal.edu.co

In [1]:
from mnist_preprocessing import preprocess_mnist_digits as mnist
from classical_neural_network import NeuralNetwork

In [2]:
x_train, y_train, x_test, y_test = mnist()

Number of filtered training examples: 12049
Number of filtered test examples: 1968
Number of unique images: 10387
Number of unique 3s:  5426
Number of unique 6s:  4912
Number of unique contradicting labels (both 3 and 6):  49

Initial number of images:  12049
Remaining non-contradicting unique images:  10338


In [3]:
neural_network = NeuralNetwork()
neural_network.train(x_train, y_train)

Epoch 1/100
324/324 [==============================] - 0s 607us/step - loss: 1.1545 - accuracy: 0.6168
Epoch 2/100
324/324 [==============================] - 0s 896us/step - loss: 0.3948 - accuracy: 0.8301
Epoch 3/100
324/324 [==============================] - 0s 1ms/step - loss: 0.3247 - accuracy: 0.8543
Epoch 4/100
324/324 [==============================] - 0s 611us/step - loss: 0.2993 - accuracy: 0.8647
Epoch 5/100
324/324 [==============================] - 0s 553us/step - loss: 0.2892 - accuracy: 0.8712
Epoch 6/100
324/324 [==============================] - 0s 554us/step - loss: 0.2846 - accuracy: 0.8768
Epoch 7/100
324/324 [==============================] - 0s 551us/step - loss: 0.2775 - accuracy: 0.8991
Epoch 8/100
324/324 [==============================] - 0s 553us/step - loss: 0.2859 - accuracy: 0.9057
Epoch 9/100
324/324 [==============================] - 0s 553us/step - loss: 0.2822 - accuracy: 0.9068
Epoch 10/100
324/324 [==============================] - 0s 545us/step - los

In [4]:
neural_network.model.evaluate(x_train, y_train)

324/324 [==============================] - 0s 543us/step - loss: 0.2629 - accuracy: 0.9082


[0.2629060745239258, 0.9082027673721313]

In [5]:
neural_network.model.evaluate(x_test, y_test)

62/62 [==============================] - 0s 602us/step - loss: 0.2780 - accuracy: 0.9202


[0.2779511511325836, 0.920223593711853]